In [1]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################
from env.quadruped.env import Env

from trajectory_planner import TrajectoryPlanner
from foot_step_planner import FootStepPlanner
from gait_scheduler import GaitScheduler
from NLP_solver import NLPSolver

from scipy.spatial.transform import Rotation
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=True, base_fix=False)
num_leg = env.num_leg
time_step = env.time_step
base_inertia = deepcopy(env.model.inertia)
base_mass = env.model.mass
friction_coef = env.friction_coef
max_force = env.model.max_force

plan_time_step = 0.05
plan_time_horizon = 20

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = False
gait_scheduler = GaitScheduler(gs_args)

tp_args = dict()
tp_args['time_step'] = plan_time_step
tp_args['time_horizon'] = plan_time_horizon
tp_args['max_accel'] = 0.1
tp_args['max_ang_accel'] = 0.1
trajectory_planner = TrajectoryPlanner(tp_args)

fsp_args = dict()
fsp_args['num_leg'] = num_leg
fsp_args['time_horizon'] = plan_time_horizon
fsp_args['abduct_org_list'] = env.model.abduct_org_list
foot_step_planner = FootStepPlanner(fsp_args)

state_dim = 12
force_dim = 12
R_mat = np.eye(force_dim)*0.0 #001
Q_mat = np.eye(state_dim)
Qf_mat = np.eye(state_dim)*10.0
nlp_args = dict()
nlp_args['R_mat'] = R_mat
nlp_args['Q_mat'] = Q_mat
nlp_args['Qf_mat'] = Qf_mat
nlp_args['time_horizon'] = plan_time_horizon
nlp_args['time_step'] = plan_time_step
nlp_args['base_inertia'] = base_inertia
nlp_args['base_mass'] = base_mass
nlp_args['friction_coef'] = friction_coef
nlp_args['max_force'] = max_force
nlp_solver = NLPSolver(nlp_args)

In [3]:
env.reset()
nlp_solver.reset()
gait_scheduler.reset()
trajectory_planner.reset()
foot_step_planner.reset()

In [4]:
c1, c2, c3, c4 = 1,2,3,4
contact_mat = np.diag([c1]*3 + [c2]*3 + [c3]*3 + [c4]*3)

In [5]:
contact_mat

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]])

In [6]:
'''
gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
gait_list[0,0] = 0.5 # offset
gait_list[0,1] = 0.5 # duration
gait_list[2,0] = 0.0 # offset
gait_list[2,1] = 0.5 # duration
gait_scheduler.gait_list = deepcopy(gait_list)
gait_scheduler.update()
'''

vel_cmd = np.zeros(3)
vel_cmd[0] = 0.0
vel_cmd[1] = 0.0
ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0
height_cmd = 0.35

In [7]:
cube = env.pybullet_client.loadURDF("cube_small.urdf")

collisionFilterGroup = 0x1
collisionFilterMask = 0x1
env.pybullet_client.setCollisionFilterGroupMask(env.planeId, -1, collisionFilterGroup, collisionFilterMask)
for i in range(env.pybullet_client.getNumJoints(env.model.sim_model)):
    env.pybullet_client.setCollisionFilterGroupMask(env.model.sim_model, i, collisionFilterGroup, collisionFilterMask)

collisionFilterGroup2 = 0x2
collisionFilterMask2 = 0x2 
for i in range(env.pybullet_client.getNumJoints(cube)):
    env.pybullet_client.setCollisionFilterGroupMask(cube, i, collisionFilterGroup2, collisionFilterMask2)

In [8]:
global_t = 0.0
mpc_cnt = 0
mpc_period = int(plan_time_step/time_step)
while global_t < 10.0:
    if mpc_cnt == 0:
        init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
        init_foot_pos_list = deepcopy(env.model.foot_pos_list)

        target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
        contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
        target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)
        state_list, force_list = nlp_solver.get_solution(init_state_list, target_trajectory_list, target_foot_step_list, contact_list)

        #'''
        for i in range(plan_time_horizon):
            time.sleep(0.1)
            state = state_list[i]
            pos_base = state[:3]
            rpy_base = state[3:6]
            vel_base = state[6:9]
            ang_vel_base = state[9:]
            orn_base = env.pybullet_client.getQuaternionFromEuler(rpy_base)

            env.pybullet_client.resetBasePositionAndOrientation(cube, pos_base, orn_base)
            env.pybullet_client.resetBaseVelocity(cube, np.zeros(3), np.zeros(3))
            #print(pos_base, target_trajectory_list[i,:3])
        #'''

    contact_list, gait_info_list = gait_scheduler.step(time_step)
    inter_weight = 1.0 - float(mpc_cnt)/mpc_period
    inter_force_list = inter_weight*force_list[0] + (1-inter_weight)*force_list[1]
    #print(inter_force_list)
    env.step(inter_force_list, contact_list, desired_foot_list=[])
    time.sleep(time_step)

    mpc_cnt = np.mod(mpc_cnt + 1, mpc_period)
    global_t += time_step
    
gait_scheduler.close()

Iteration limit reached    (Exit mode 9)
            Current function value: 0.11353016593033127
            Iterations: 50
            Function evaluations: 258
            Gradient evaluations: 50
Iteration limit reached    (Exit mode 9)
            Current function value: 7.9345054479947486
            Iterations: 50
            Function evaluations: 383
            Gradient evaluations: 50
Iteration limit reached    (Exit mode 9)
            Current function value: 41.388068054590796
            Iterations: 50
            Function evaluations: 420
            Gradient evaluations: 50
Iteration limit reached    (Exit mode 9)
            Current function value: 71.66402444928896
            Iterations: 50
            Function evaluations: 437
            Gradient evaluations: 50
Iteration limit reached    (Exit mode 9)
            Current function value: 57.09988826136883
            Iterations: 50
            Function evaluations: 447
            Gradient evaluations: 50
Iteration l

KeyboardInterrupt: 